In [ ]:
from nems_lbhb.baphy_experiment import BAPHYExperiment
import charlieTools.baphy_remote as br
import charlieTools.noise_correlations as nc
from charlieTools.plotting import compute_ellipse
from sklearn.decomposition import PCA
import nems.db as nd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

### Per-trial PSTHs
* plot PSTH per trial for each behavior condition, aligned to catch, aligned to target, aligned to trial start. 

#### Options

In [ ]:
site = 'CRD010b'
options = {'resp': True, 'pupil': False, 'rasterfs': 100}

trial_outcomes = ['HIT_TRIAL', 'MISS_TRIAL', 'CORRECT_REJECT_TRIAL', 'PASSIVE_EXPERIMENT']
# could also do: FALSE_ALARM_TRIAL, INCORRECT_HIT_TRIAL

# trial window (so can extract custom slices of data)
trial_window = options['rasterfs'] * 4  # 4 seconds of data

#### Load recording

In [ ]:
# get parmfiles
sql = "SELECT sCellFile.cellid, sCellFile.respfile, gDataRaw.resppath from sCellFile INNER JOIN" \
            " gCellMaster ON (gCellMaster.id=sCellFile.masterid) INNER JOIN" \
            " gDataRaw ON (sCellFile.rawid=gDataRaw.id)" \
            " WHERE gCellMaster.siteid=%s" \
            " and gDataRaw.runclass='TBP' and gDataRaw.bad=0"
d = nd.pd_query(sql, (site,))
d['parmfile'] = [f.replace('.spk.mat', '.m') for f in d['respfile']]
parmfiles = np.unique(np.sort([os.path.join(d['resppath'].iloc[i], d['parmfile'].iloc[i]) for i in range(d.shape[0])])).tolist()
manager = BAPHYExperiment(parmfiles)
rec = manager.get_recording(**options)
rec['resp'] = rec['resp'].rasterize()

#### Plot PSTHs
* For each trial outcome, plot:
    * per-neuron trial psth as heatmap
    * average across neuron psth (pop. psth)
    * psth along PC1
    * along PC2
    * in state-space (PC1 vs. PC2)

In [ ]:
rt = rec.copy()
rt = rt.and_mask('TRIAL')
for to in trial_outcomes:

    fig = plt.figure(figsize=(14, 8))
    pn = plt.subplot2grid((3, 5), (0, 0), rowspan=3, colspan=2)
    pop = plt.subplot2grid((3, 5), (0, 2), colspan=3)
    pc1 = plt.subplot2grid((3, 5), (1, 2), colspan=3)
    pc2 = plt.subplot2grid((3, 5), (2, 2), colspan=3)

    r = rt.copy()
    r = r.and_mask(to)

    # single neuron psth
    psth = r['resp'].extract_epoch('TRIAL', mask=r['mask'])
    psth = np.nanmean(psth, axis=0)
    pn.imshow(psth, aspect='auto', cmap='Reds')
    pn.set_ylabel('Neuron')
    pn.set_xlabel(f"Trial time (binned at {options['rasterfs']} Hz)")



    pn.set_title(to, fontsize=14)

    fig.tight_layout()
